In [42]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

# -------------------------------------------------------------------
# 1) Load raw data (do not modify df_raw)
# -------------------------------------------------------------------
df_raw = pd.read_csv("../data_raw/yemen-core-dataset-health.csv")

# Quick inventory checks (run these when you want to inspect)
df_raw.shape
df_raw.columns
df_raw.dtypes

# Work on a copy so the raw dataframe remains untouched
df = df_raw.copy()

# -------------------------------------------------------------------
# 2) Clean list-like string fields (e.g., "['Abyan']" -> "Abyan")
# -------------------------------------------------------------------
def clean_list_string(x):
    """Convert list-like strings such as "['Abyan']" into 'Abyan'."""
    if pd.isna(x):
        return x
    s = str(x).strip()
    if s.startswith("[") and s.endswith("]"):
        s = s[1:-1].strip()          # remove [ ]
        s = s.strip("'").strip('"')  # remove surrounding quotes
    return s

list_like_cols = [
    "unocha_operational_hub",
    "governorate", "district",
    "pcode_1", "pcode_2",
    "gid_1", "gid_2"
]

for c in list_like_cols:
    df[c] = df[c].apply(clean_list_string)

# Verify cleaning: should show clean values (no brackets)
df[list_like_cols].head()

# QC check: count any values that still look list-like
for c in list_like_cols:
    still_listy = df[c].astype(str).str.strip().str.startswith("[").sum()
    print(c, still_listy)

# -------------------------------------------------------------------
# 3) Parse dates correctly (raw format is DD/MM/YYYY) and standardize month
# -------------------------------------------------------------------
df["date"] = pd.to_datetime(df_raw["date"], format="%d/%m/%Y", errors="coerce")

# QC check: confirm date range + missing count
df["date"].min(), df["date"].max(), df["date"].isna().sum()

# Standardize to month-start timestamp for consistent monthly grouping
df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()

# Create district-month key (useful for sanity checks and debugging)
df["district_month_key"] = df["pcode_2"].astype(str) + "_" + df["month"].astype(str)

# QC check: month range + unique keys vs total rows
df["month"].min(), df["month"].max(), df["district_month_key"].nunique(), df.shape[0]

# -------------------------------------------------------------------
# 4) Identify duplicates at the district-month level (pcode_2 + month)
# -------------------------------------------------------------------
dup_mask = df.duplicated(subset=["pcode_2", "month"], keep=False)
dup_mask.sum()

# Inspect a sample of duplicates to understand what differs
df.loc[dup_mask, [
    "pcode_2", "district", "governorate", "month",
    "percent_functioning_health_centres",
    "num_suspected_cases_cholera", "num_deaths_cholera",
    "percent_attack_rate", "num_health_workers"
]].sort_values(["pcode_2", "month"]).head(30)

# Flag rows involved in duplicates (this is row-level at this stage)
df["was_duplicated"] = dup_mask
df["was_duplicated"].value_counts()

# -------------------------------------------------------------------
# 5) Deduplicate to one row per district-month using documented rules
#    - counts: MAX
#    - rates/percentages: MEDIAN
# -------------------------------------------------------------------
agg_rules = {
    # Percentage / rate variables → MEDIAN
    "percent_functioning_health_centres": "median",
    "percent_attack_rate": "median",
    "percent_individuals_health_facilities_problems": "median",

    # Count variables → MAX
    "num_suspected_cases_cholera": "max",
    "num_deaths_cholera": "max",
    "num_health_workers": "max",

    # Flag → MAX (True if any duplicate existed in that district-month)
    "was_duplicated": "max",
}

group_cols = [
    "pcode_2", "month",
    "district", "governorate",
    "pcode_1", "gid_1", "gid_2",
    "unocha_operational_hub"
]

df_dedup = (
    df
    .groupby(group_cols, as_index=False)
    .agg(agg_rules)
)

# QC check: after deduplication, there should be 0 district-month duplicates
df_dedup.duplicated(subset=["pcode_2", "month"]).sum()

# QC check: duplicated flag should now reflect district-month level (True ≈ 22)
df_dedup["was_duplicated"].value_counts()

# -------------------------------------------------------------------
# 6) Coverage diagnostics (district reporting over time)
# -------------------------------------------------------------------
coverage_by_month = (
    df_dedup.groupby("month")["pcode_2"]
    .nunique()
    .reset_index(name="num_districts_reporting")
)

coverage_by_month.head(), coverage_by_month.tail()

# -------------------------------------------------------------------
# 7) Missingness diagnostics (by column)
# -------------------------------------------------------------------
missingness = (
    df_dedup.isna()
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

missingness.columns = ["column", "missing_fraction"]
missingness["missing_percent"] = (missingness["missing_fraction"] * 100).round(1)
missingness

# -------------------------------------------------------------------
# 8) Export processed outputs (filenames unchanged)
# -------------------------------------------------------------------
df_dedup.to_csv("../data_processed/yemen_health_cleaned_deduped.csv", index=False)

coverage_by_month.to_csv("../data_processed/qc_coverage_by_month.csv", index=False)

missingness.to_csv("../data_processed/qc_missingness_by_column.csv", index=False)






unocha_operational_hub 0
governorate 0
district 0
pcode_1 0
pcode_2 0
gid_1 0
gid_2 0
